# Clustering: LDA w/ sklearn
Prototyping NB for Clustering EUvsVirus Hackathon projects based on LDA using sklearn.

In [1]:
import os, warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd

import spacy
from spacymoji import Emoji
import regex
import emoji

%matplotlib inline

## Load

Data

In [2]:
data_path = "/mnt/DATA/RWData Dropbox/EUvsVirus"
data_fpath = os.path.join(data_path, "all_data.tsv")

In [3]:
data = pd.read_csv(data_fpath, sep="\t")
print(data.info())
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Challenge     2159 non-null   object
 1   SubChallenge  2159 non-null   object
 2   ProjURL       2159 non-null   object
 3   title         2159 non-null   object
 4   text          2069 non-null   object
dtypes: object(5)
memory usage: 84.5+ KB
None


(2159, 5)

NLP Model

In [4]:
nlp = spacy.load("en_core_web_md", disable=["ner", "parser", "tagger"])
emoji = Emoji(nlp, merge_spans=False)
nlp.add_pipe(emoji, first=True)

## Preprocess

In [5]:
data_pure = data.dropna(subset=["text"]) # remove text nan rows
raw_corpus = data_pure.text

### Cleansing

In [6]:
# remove \n \r \t
corpus = raw_corpus.str.replace(r"[\n\r\t]", " ")

# maybe later:
# remove header texts
# split and structure text parts
# etc

In [7]:
def spacy_cleaner(doc):
    tokens = [token for token in nlp(doc)]
    cleaned_tokens = []
                                        
    for token in tokens:
        if not any(map(lambda x: getattr(token, x), ["is_punct", "is_digit",
                                                     "is_space", "like_url",
                                                     "like_num", "like_email",
                                                     "is_stop"
                                                     ])):
            # maybe lemmatize here
            cleaned_tokens.append(token)
    
    return " ".join([token.orth_ for token in cleaned_tokens])

cleaned_corpus = corpus.apply(spacy_cleaner)

## LDA